# Seminario de Ciencia de Datos

## 2º Cuatrimestre de 2017

## Módulo 2 - Series Temporales

### Integrantes:

- Mascitti, Julio Augusto - 954/11 - mascittija@gmail.com

- Aleman, Damián - ###/## - usuario@dominio

- Guerson, Matias Carlos - 925/10 - matias.guerson@gmail.com

#### Introducción

En este trabajo tenemos por objetivo trabajar con series temporlaes y grandes volúmenes de datos. Para lo mismo la catedra nos porporcionó un set de datos de Electroencefalogramas (EGGs) de 20 pacientes divididos en 2 grupos distinguibles. Estos datos fueron recolectados de mediciones realizadas por 256 electrodos ubicados en un casco que se conecta al paciente. El experimento conciste en estimular a los distintos pacientes conectados a los cascos para poder medir su reacción ante dicho estímulo.
Para más detalles leer el enunciado, que cuenta con información sobre la técnica de electroencefalografía y cuenta con detalles sobre la implementación del experimento.

### Analisis preliminar de los datos

In [ ]:
import pandas as pd
import numpy as np
from scipy import signal
from scipy.stats import entropy
import scipy.io as io
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
matplotlib.style.use('ggplot')

Importando los datos del primer paciente para familiarizarnos un poco con la información que disponemos

In [ ]:
# path = "../../../../"
path = "./"

data = io.loadmat('{}data/P01.mat'.format(path))

matriz = data['data']
print(matriz.shape)
print(len(matriz))   # x = epochs
print(len(matriz[0]))   # y = electrodos
print(len(matriz[0][0]))   # z = muestras 

Disponemos de una matriz por paciente con 894 Epochs, 256 electrodos y 201 muestras

In [ ]:
electrodo8 = matriz[:,7,:]
print(electrodo8.shape)
print(electrodo8)

El experimento conciste en estimular al paciente y ver su reacción.

La grabación continua de 30 minutos es cortada tomando como parámetro el inicio de la estimulación auditiva (T0). Cada segmento (denominado epoch o trial) tiene una duración de 1.54 segundos y comienza 200 ms antes de T0. En otras palabras, cada epoch comienza en -0.200s y termina en 1.34s donde 0s es la ocurrencia de un estímulo auditivo.

Algo interesante para ver, puede ser el promedio de los voltajes de los epochs para las 201 mediciones que tiene cada epoch para un electrodo en particular

In [1]:
epoch_promedio = electrodo8.mean(axis=0)
plt.plot(epoch_promedio)

NameError: name 'electrodo8' is not defined

Calculemos Welch al Epoch_Promedio y veamos que podemos observar

In [ ]:
frec, power = signal.welch(x= epoch_promedio,fs = 250)
print(frec)
plt.plot(frec, power)

Podemos apreciar un pico alrededor de lo 10 Hz

## Análisis de frecuencias

- Calcular la media entre los electrodos 8, 44, 80, 131 y 185 (el primer electrodo es el 0) y realizar una figura que muestre las frecuencias en el eje Y, los epochs en el eje X, y la potencia usando una escala de color.

In [ ]:
from matplotlib.colors import LogNorm


electrodo44 = matriz[:,43,:]
electrodo80 = matriz[:,79,:]
electrodo131 = matriz[:,130,:]
electrodo185 = matriz[:,184,:]

elec_promedio = np.array([electrodo8,electrodo44,electrodo80,electrodo131,electrodo185]).mean(axis=0)
print(elec_promedio.shape)


welchs = np.zeros((894,2,101))
for i_epoch in range(len(elec_promedio)):
    welchs[i_epoch] = signal.welch(x=elec_promedio[i_epoch], fs=250)
print(welchs.shape)    

#plt.plot(welchs[0,0,0:42],welchs[0,1,0:42])
#print(welchs[0,1,:42])

potencias = welchs[:,1,:42]


In [ ]:
#sns.heatmap(potencias.T, norm=LogNorm(vmin=1.0e-14, vmax=1.0e-08))
#sns.plt.yticks(np.linspace(0,42,6), ['a','b','c','d','e','f'])
#sns.plt.xticks(np.linspace(100,800))
#sns.plt.show()
#sns.heatmap(potencias.T)

plt.axes([1,1,1,1])
plt.grid(False)
yticks = np.linspace(0,len(potencias[0]),10, dtype=np.int)
plt.yticks(yticks, welchs[0,0,yticks])
plt.imshow(potencias.T, interpolation='nearest', cmap='hot', origin='lower', aspect='auto', norm=LogNorm(vmin=1.0e-14, vmax=1.0e-08))
plt.colorbar(shrink=.92)

plt.show()

- Calcular la potencia media (entre epochs) para cada frecuencia y graficar la potencia en funcion de la frecuencia para cada canal.


In [ ]:
welchs_by_elect_and_epoch = np.empty((256,894,2,101))

for i_epoch in range(len(matriz)):
    epoch = matriz[i_epoch]
    for i_elect in range(len(epoch)):
        f,p = signal.welch(x=epoch[i_elect], fs=250)
        welchs_by_elect_and_epoch[i_elect,i_epoch] = np.array([f,p])

In [ ]:
#print(welchs_by_elect_and_epoch.shape)
power_mean_between_epochs = welchs_by_elect_and_epoch.mean(axis=1)
#print(power_mean_between_epochs.shape)

for elect in power_mean_between_epochs:
    plt.plot(elect[0,:42], elect[1,:42])

plt.show()

- Calcular los valores de cada banda de frecuencia, promediados entre los electrodos (todos) y epochs para cada sujeto.

In [ ]:
ranges = [0,4,8,13,30,45]
acums = np.zeros((5))
counts = np.zeros((5))

for i_band in range((len(ranges)-1)):
    for elect in welchs_by_elect_and_epoch:
        for epoch in elect:
            for welch in range(len(epoch[0])):
                if (epoch[0, welch] >= ranges[i_band]) and (epoch[0, welch] < ranges[i_band+1]):
                    acums[i_band] += epoch[1, welch]
                    counts[i_band] += 1
            

                
print(acums)
print(counts)

In [ ]:
bands_means = acums / counts
bands_means

In [ ]:
# Alpha
bands_means[2] 

- Tomar la potencia de cada sujeto en la banda Alpha y graficar cada uno de los graficos categóricos de seaborn.

¿Cuál recomendaría para graficar y comparar distribuciones?

In [ ]:
def calculate_welchs(data):
    welchs_by_elect_and_epoch = np.empty((256,len(data),2,101))

    for i_epoch in range(len(data)):
        epoch = data[i_epoch]
        for i_elect in range(len(epoch)):
            f,p = signal.welch(x=epoch[i_elect], fs=250)
            welchs_by_elect_and_epoch[i_elect,i_epoch] = np.array([f,p])
    
    return welchs_by_elect_and_epoch

In [ ]:
def calculate_power_mean(welchs_by_elect_and_epoch, frequency_range):
    freqs = welchs_by_elect_and_epoch[0,0,0,:]
    alphas_idxs = np.where((freqs>=frequency_range[0]) * (freqs<frequency_range[1]))
    alphas = welchs_by_elect_and_epoch[:,:,1,alphas_idxs]
    alphas = alphas[:,:,0,:]
    return alphas.mean()

In [ ]:
for t in ['P','S']:
    for i in range(1,11):
        try:
            data_file = path.join('../../../../data/',t) + "%02d" % (i) + '.mat'
            welch_file = 'welch_' + t + "%02d" % (i) + '.dmp'
            print(data_file)
            print(welch_file)
            data = io.loadmat(data_file)['data']
            print(data.shape)
            welch = calculate_welchs(data)
            welch.dump(welch_file)
        except Exception as exception:
            print(str(exception))
            pass


In [ ]:
def load_welchs_and_calculate_power_means(frequency_range):

    power_means = np.zeros((2,10))

    for t in ['P','S']:
        for i in range(1,11):
            try:
                welch_file = 'welch_' + t + "%02d" % (i) + '.dmp'
                print(welch_file)
                data = np.load(welch_file)
                print(data.shape)
                power_mean = calculate_power_mean(data, frequency_range)
                row = 0 if(t == 'P') else 1
                power_means[row, i-1] = power_mean
            except Exception as exception:
                print(str(exception))
                pass

    print(repr(power_means))
    return power_means

In [ ]:
frequencies_ranges = {"delta": (0,4), "theta": (4,8), "alpha": (8,13), "beta": (13,30), "gamma": (30,125)}

for name,frequency_range in frequencies_ranges.items():
    frequency_power_means = load_welchs_and_calculate_power_means(frequency_range)
    frequency_power_means.dump(name + 'power_means.dmp')


In [ ]:
alpha = np.load("alphapower_means.dmp").T
beta = np.load("betapower_means.dmp").T
gamma = np.load("gammapower_means.dmp").T
delta = np.load("deltapower_means.dmp").T
theta = np.load("thetapower_means.dmp").T

power_sum = alpha + beta + gamma + delta + theta

alpha_norm = alpha / power_sum
beta_norm = beta / power_sum
gamma_norm = gamma / power_sum
delta_norm = delta / power_sum
theta_norm = theta / power_sum

df = pd.DataFrame(alpha, columns=list('PS'))
df

### Stripplot

In [ ]:
ax = sns.stripplot(data=df)
ax.set_autoscaley_on(True)
ax.set_ylim(1e-14,6e-12)
sns.plt.show()

### Swarmplot

In [ ]:
ax = sns.swarmplot(data=df)
ax.set_autoscaley_on(True)
ax.set_ylim(1e-14,6e-12)
sns.plt.show()

### Boxplot

In [ ]:
ax = sns.boxplot(data=df)
ax.set_autoscaley_on(True)
sns.plt.show()

### Violinplot

In [ ]:
ax = sns.violinplot(data=df)
ax.set_autoscaley_on(True)
sns.plt.show()

### Lvplot

In [ ]:
ax = sns.lvplot(data=df)
ax.set_autoscaley_on(True)
sns.plt.show()

### Pointplot

In [ ]:
ax = sns.pointplot(data=df)
ax.set_autoscaley_on(True)
sns.plt.show()

### Barplot

In [ ]:
ax = sns.barplot(data=df)
ax.set_autoscaley_on(True)
sns.plt.show()

Pensamos que el que más información aporta a simple vista sin entrar en muchos detalles es el ViolinPlot. Por lo tanto los proximos puntos vamos a realizarlos con este gráfico.

- Para cada banda de frecuencia, graficar según lo elegido en el punto c) y realizar un test estadístico apropiado.

In [ ]:
def plot_violin(df):
    ax = sns.violinplot(data=df)
    ax.set_autoscaley_on(True)
    sns.plt.show()
    
def test_same_mean(data1, data2):
    t, p_value = sp.stats.ttest_rel(data1, data2)
    print(p_value)
    return "No hay evidencia suficiente para rechazar la hipotesis nula" if (p_value > 0.05) else "Se rechaza la hipotesis nula"

#### Banda Delta (Delta < 4 Hz)

In [ ]:
df = pd.DataFrame(delta, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

df = pd.DataFrame(delta_norm, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

#### Banda Theta (4 Hz <= Theta < 8 Hz)

In [ ]:
df = pd.DataFrame(theta, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

df = pd.DataFrame(theta_norm, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

#### Banda Beta (13 Hz <= Beta < 30 Hz)

In [ ]:
df = pd.DataFrame(beta, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

df = pd.DataFrame(beta_norm, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

#### Banda Gamma (30 Hz <= Gamma < Nyquist)

In [ ]:
df = pd.DataFrame(gamma, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

df = pd.DataFrame(gamma_norm, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

## Análisis de Información

- Computar una medida de información intra-electrodo. Calcular la media entre canales y epochs para cada sujeto. Realizar el gráfico elegido en el punto c) de la sección anterior, acompañado del test estadístico apropiado.

In [ ]:
pows = []
freqs = []
pacientes = [1]
for elect in welchs_by_elect_and_epoch:
    for epoch in elect:
        for welch in range(len(epoch[0])):
            if (epoch[0, welch] >= 8) and (epoch[0, welch] < 13):
                freqs.append(epoch[0, welch])
                pows.append(epoch[1, welch])


In [ ]:
from scipy.stats import binned_statistic
# calculo el electrodo promedio
elec = matriz[:,1,:]

#discritizo los valores de amplitud (voltajes) en bins mediante la regla de Freedman-Diaconis.
bins =  (np.histogram(elec, bins='fd',density=False))
# bins tiene las cantidades de cada amplitud en los distintos bins [0] y las respuectivas frecuencias asociadas
#bins
#entropy(bins[0])
bins

In [ ]:
def cantidad_informacion(elec):
    bins =  (np.histogram(elec, bins='fd',density=False)) 
    return entropy(bins[0])

# cantidad_informacion(matriz[:,1,:].mean(axis=0))

In [ ]:

def cantidad_informacion_sujeto(matriz):
    cant_inf = []
    for i in range(matriz.shape[1]):
        cant_inf.append(cantidad_informacion(matriz[:,i,:]))
# plt.plot(cant_inf)
    return np.array(cant_inf).mean()


#plot de todas las cantidades de informacion por electrodo

In [ ]:
mediasP = []
mediasS = []
for l in ['P','S']:
    for i in range(1,11):
        dat = io.loadmat('{0}data/{1}%02d.mat'.format(path,l) %i)
        mat = dat['data']
        media = cantidad_informacion_sujeto(mat)
        if l=='P':
            mediasP.append(media)
        else:
            mediasS.append(media)

In [ ]:
medias = pd.DataFrame({"P":mediasP,"S":mediasS})

In [ ]:
df = pd.DataFrame(medias, columns=list('PS'))
plot_violin(df)

print(test_same_mean(df['P'], df['S']))

In [ ]:
electrodo44 = matriz[:,43,:]
electrodo80 = matriz[:,79,:]
electrodo131 = matriz[:,130,:]
electrodo185 = matriz[:,184,:]

infs = []
infs.append(cantidad_informacion(electrodo44))
infs.append(cantidad_informacion(electrodo80))
infs.append(cantidad_informacion(electrodo131))
infs.append(cantidad_informacion(electrodo185))
plt.plot(infs)

In [ ]:
#calcular la media entre canales y epochs

sns.stripplot(media)
matriz.nbytes

In [ ]:
mediasP = []
mediasS = []
for l in ['P','S']:
    for i in range(1,11):
        dat = io.loadmat('{0}data/{1}%02d.mat'.format(path,l) %i)
        mat = dat['data']
        media = mat.mean(axis=0).mean(axis=0)
        if l=='P':
            mediasP.append(media)
        else:
            mediasS.append(media)



In [ ]:
ax =sns.stripplot(y=mediasP[0]) # habria que agregar la de los otros pacientes
ax.set_yscale('log')
ax.set_ylim(1e-10,1e-7)

In [ ]:
np.histogram(elec1)[0]

- Repetir el punto a) para una medida inter-electrodo.

In [ ]:
def informacion_mutua(elec1, elec2):
    bins_pares = np.histogram2d(elec1.flatten(), elec2.flatten())[0]
    bins_elec1  = np.histogram(elec1, bins='fd')[0]
    bins_elec2  = np.histogram(elec2, bins='fd')[0]
    
    H_xy = entropy(bins_pares.flatten())
#     print(str(H_xy),"entropia conjunta")
    H_x  = entropy(bins_elec1)
#     print(str(H_x),"entropia x")
    H_y  = entropy(bins_elec2)
#     print(str(H_y),"entropia y")
    
    return H_x + H_y - H_xy

In [ ]:
# calculo la medida de informacion inter electrodo
cant_inf2d = []
for i in [43,79,130,184]:
    for j in [43,79,130,184]:
        if i!=j:
            cant_inf2d.append(informacion_mutua(matriz[:,i,:],matriz[:,j,:]))
plt.plot(cant_inf2d)

In [ ]:
cantidad_informacion2d(matriz[:,5,:],matriz[:,6,:])